In [ ]:
import pandas as pd

df = pd.read_parquet("data/driver_stats.parquet")
df.sort_values("event_timestamp")

In [ ]:
from datetime import datetime
import pandas as pd

from feast import FeatureStore

# Note: see https://docs.feast.dev/getting-started/concepts/feature-retrieval for 
# more details on how to retrieve for all entities in the offline store instead
entity_df = pd.DataFrame.from_dict(
    {
        # entity's join key -> entity values
        "driver_id": [1001, 1002, 1003],
        # "event_timestamp" (reserved key) -> timestamps
        "event_timestamp": [
            datetime(2021, 4, 12, 10, 59, 42),
            datetime(2021, 4, 12, 8, 12, 10),
            datetime(2021, 4, 12, 16, 40, 26),
        ],
        # (optional) label name -> label values. Feast does not process these
        "label_driver_reported_satisfaction": [1, 5, 3],
        # values we're using for an on-demand transformation
        "val_to_add": [1, 2, 3],
        "val_to_add_2": [10, 20, 30],
    }
)

store = FeatureStore(repo_path=".")

training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips"
    ],
).to_df()

print("----- Feature schema -----\n")
training_df.info()

print()
print("----- Example features -----\n")
training_df.head()

In [ ]:
# Using feature service 
store = FeatureStore(repo_path="./feature_repo")
feature_service = store.get_feature_service("driver_activity_v1")
features = store.get_historical_features(
    features = feature_service,
    entity_df=entity_df
).to_df()
features.head()

## Online scoring
Before running the cell bellow we need to move the data to online feature store using the materialisaiton. 

```{bash}
CURRENT_TIME=$(date -u +"%Y-%m-%dT%H:%M:%S")
feast materialize-incremental $CURRENT_TIME
```

In [ ]:
from pprint import pprint
from feast import FeatureStore

store = FeatureStore(repo_path=".")

feature_vector = store.get_online_features(
    features=[
        "driver_hourly_stats2:conv_rate",
        "driver_hourly_stats2:acc_rate",
        "driver_hourly_stats2:avg_daily_trips",
    ],
    entity_rows=[
        # {join_key: entity_value}
        {"driver_id": 1004},
        {"driver_id": 1005},
    ],
).to_dict()

pprint(feature_vector)

# UI
Check UI with 
```
feast ui
```